# Fractures

This notebook is intended to experiment with fractures in a simple domain with simple boundary conditions.

Some central questions to be asked:
* How are solutions affected by high `aperture * fracture_perm` relative to rock permeability?
* What about low?
* What is the relationship between fracture aperature and fracture permeability? Look up `cubic law`.


## Basic setup

This notebook will consider a 2D domain with physical dimensions of $[0,1]^2$. As such, fractures are 1D lines, and interesctions of fractures are 0D points.

Let $\Omega$ be a unit square with boundary $\partial\Omega$. Consider a partition of the boundary in two parts consisting of $\partial\Omega_d$, representing Dirichlet boundary, and $\partial\Omega_n$, representing a Neumann boundary. By convention, let $\mathbf{n}$ be the outward unit vector normal to $\partial\Omega$.

Single phase flow, in its primal formulation, can be written as
$$-\nabla \cdot K \nabla p = f$$
with boundary conditions on $\partial\Omega_n$ and $\partial\Omega_d$:
$$\mathbf{u}\cdot\mathbf{n}=u_b, \quad p = p_b$$
Here, $f$ is a scalar source, $K$ is the permability tensor, $p_b$ is the pressure at the boundary (Dirichlet), and $u_b$ us the flux at the boundary (Neumann).

## Import necessary modules
Before we can start, we need the following modules:

In [1]:
import numpy as np
import scipy.sparse as sps
import porepy as pp

## Define the fractured grid
Next, we define a $[0,1]^2$ grid with $20\cdot20$ cells. In addition, a fracture located at $y=0.8$ parallel to the x-axis from $x=0.2$ to $x=0.8$ is defined.

This setup is passed along to a method `cart_grid` (note, not the same as `pp.CartGrid`!) which generates the grid containing a meshed 2D region (rock domain) and a 1D line representing the fracture. The communication between the fracture and the rock domain is handled by interfaces. 

In [13]:
Nx = Ny = 20
phys_dims = [1,1]
fracture = np.array([[0.2, 0.8], [0.8, 0.8]])
gb = pp.meshing.cart_grid([fracture], [Nx, Ny], physdims=phys_dims)

data_key="flow"

Next, define the permeability and boundary conditions of each grid in the grid bucket. In our case, we have one grid representing the rock domain and one representing the fracture.

In [20]:
for g,d in gb:
    b_faces = g.tags['domain_boundary_faces'].nonzero()[0]
    print(g.face_centers[1,b_faces])
    print("")
    print(b_faces)
    print("------")

[0.025 0.025 0.075 0.075 0.125 0.125 0.175 0.175 0.225 0.225 0.275 0.275
 0.325 0.325 0.375 0.375 0.425 0.425 0.475 0.475 0.525 0.525 0.575 0.575
 0.625 0.625 0.675 0.675 0.725 0.725 0.775 0.775 0.825 0.825 0.875 0.875
 0.925 0.925 0.975 0.975 0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
 1.    1.    1.    1.    1.    1.    1.    1.   ]

[  0  20  21  41  42  62  63  83  84 104 105 125 126 146 147 167 168 188
 189 209 210 230 231 251 252 272 273 293 294 314 315 335 336 356 357 377
 378 398 399 419 420 421 422 423 424 425 426 427 428 429 430 431 432 433
 434 435 436 437 438 439 820 821 822 823 824 825 826 827 828 829 830 831
 832 833 834 835 836 837 838 839]
------
[]

[]
------


In [24]:
# Thickness of fracture
aperture = 1e-3
fracture_perm = 1e-5

for g, d in gb:
    # The concept of specific volumes accounts for the thickness
    # of the fracture, which is collapsed in the mixed-dimensional
    # model.
    specific_volumes = np.power(aperture, gb.dim_max()-g.dim)
    # Permeability
    k = np.ones(g.num_cells) * specific_volumes
    if g.dim < gb.dim_max():
        k *= fracture_perm
    perm = pp.SecondOrderTensor(k)                     

    # Unitary scalar source already integrated in each cell
    f = 1e-2* g.cell_volumes * specific_volumes 

    # Boundary conditions
    b_faces = g.tags['domain_boundary_faces'].nonzero()[0]
    bc = pp.BoundaryCondition(g, b_faces, ['dir']*b_faces.size)
    bc_val = np.zeros(g.num_faces)
    bc_val[b_faces] = g.face_centers[1, b_faces]

    parameters = {"second_order_tensor": perm, "source": f, "bc": bc, "bc_values": bc_val}
    pp.initialize_data(g, d, data_key, parameters)

for e, d in gb.edges():
        gl, _ = gb.nodes_of_edge(e)
        mg = d["mortar_grid"]
        # Division through aperture/2 may be thought of as taking the gradient, i.e.
        # dividing by the distance from the matrix to the center of the fracture.
        kn = fracture_perm / (aperture/2)
        pp.initialize_data(mg, d, data_key, {"normal_diffusivity": kn})

Let's make note of some features of the above code.

`specific_volumes` is either equal to the fracture aperture if a 1D fracture is being considered, or is $1$ if the rock domain is considered. (If a 0D point is considered, `specific_volumes` will be the aperture squared).

For fractures, the permeability tensor which initially is homogeneously 1, is multiplied by the `fracture_perm` only when a fracture is considered. This is similar for the source term $f$.

